In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime, calendar
import scipy.stats as stats
import statsmodels.api as sm
import re
#from tensorflow.keras import layers, models
import warnings
warnings.filterwarnings(action='ignore')
%matplotlib inline
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False


#본사출장자수 scaling 어떻게 할지 정하기
#강수량 등도 어떻게 정규화할지 정하기
#여러 feature등 다듬기

In [67]:
def k_fold_CrossValidation(train_x, train_y, num, models, weights, reg=True, random_state=42, minmax=False):
    
    ##모델은 반드시 [model1, model2, ...]의 리스트형식이어야 합니다.
    ##모델을 한 개만 줄 시 [models], [weights] 형식이어야 함.
    n_models = len(models)
    
    if sum(weights) > 1.:
        return print("weight의 합이 1을 초과합니다.")
    if sum(weights) != 1. and reg==False:
        return print('reg=False인데, weight합이 1이 아닙니다.')
    
    n_weights = len(weights)
    if n_models != n_weights:
        return print("weight와 model의 개수가 다릅니다.")
    
    
    kf = KFold(n_splits=num, random_state=random_state)
    kf.get_n_splits(train_x)
    mae = []
    
    for train_index, test_index in kf.split(train_x):
        
        X_train = copy.deepcopy(train_x.iloc[train_index])
        X_valid = copy.deepcopy(train_x.iloc[test_index])
        Y_train = copy.deepcopy(train_y.iloc[train_index])
        Y_valid = copy.deepcopy(train_y.iloc[test_index])
        
        try:
            X_train['강수량'] = X_train['점심강수량']
            X_valid['강수량'] = X_valid['점심강수량']
            del X_train['점심강수량'], X_valid['점심강수량']
        except:
            X_train['강수량'] = X_train['저녁강수량']
            X_valid['강수량'] = X_valid['저녁강수량']
            del X_train['저녁강수량'], X_valid['저녁강수량']
        
        
        
        if minmax == False:
            
            to_bias = X_train['본사출근자수'].mean()
            to_scale = X_train['본사출근자수'].std()
            covid_bias = X_train['Covid19'].mean()
            covid_scale = X_train['Covid19'].std()
            dust10_bias = X_train['PM10'].mean()
            dust10_scale = X_train['PM10'].std()
            #holy_bias = X_train['본사휴가자수'].mean()
            #holy_scale = X_train['본사휴가자수'].std()
            #out_bias = X_train['본사출장자수'].mean()
            #out_scale = X_train['본사출장자수'].std()
            TO_bias = X_train['본사정원수'].mean()
            TO_scale = X_train['본사정원수'].std()
            #excess_bias = X_train['본사시간외근무명령서승인건수'].mean()
            #excess_scale = X_train['본사시간외근무명령서승인건수'].std()
            #home_bias = X_train['현본사소속재택근무자수'].mean()
            #home_scale = X_train['현본사소속재택근무자수'].std()
            rain_bias = X_train['강수량'].mean()
            rain_scale = X_train['강수량'].std()

            
        else:
            to_bias = X_train['본사출근자수'].min()
            to_scale = X_train['본사출근자수'].max() - X_train['본사출근자수'].min()
            covid_bias = X_train['Covid19'].min()
            covid_scale = X_train['Covid19'].max() - X_train['Covid19'].min()
            dust10_bias = X_train['PM10'].min()
            dust10_scale = X_train['PM10'].max() - X_train['PM10'].min()
            TO_bias = X_train['본사정원수'].min()
            TO_scale = X_train['본사정원수'].max() - X_train['본사정원수'].min()
            rain_bias = X_train['강수량'].min()
            rain_scale = X_train['강수량'].max() - X_train['강수량'].min()
            
        if to_scale != 0:
            X_train['본사출근자수'] = (X_train['본사출근자수'] - to_bias) / to_scale
            X_valid['본사출근자수'] = (X_valid['본사출근자수'] - to_bias) / to_scale
        

        if covid_scale != 0:
            X_train['Covid19'] = (X_train['Covid19'] - covid_bias) / covid_scale
            X_valid['Covid19'] = (X_valid['Covid19'] - covid_bias) / covid_scale
            
        if dust10_scale != 0:
            X_train['PM10'] = (X_train['PM10'] - dust10_bias) / dust10_scale
            X_valid['PM10'] = (X_valid['PM10'] - dust10_bias) / dust10_scale            
            
        #if holy_scale != 0:
        #    X_train['본사휴가자수'] = (X_train['본사휴가자수'] - holy_bias) / holy_scale
        #    X_valid['본사휴가자수'] = (X_valid['본사휴가자수'] - holy_bias) / holy_scale
        #
        #if out_scale != 0:
        #    X_train['본사출장자수'] = (X_train['본사출장자수'] - out_bias) / out_scale
        #    X_valid['본사출장자수'] = (X_valid['본사출장자수'] - out_bias) / out_scale
        #
        if TO_scale != 0:
            X_train['본사정원수'] = (X_train['본사정원수'] - TO_bias) / TO_scale
            X_valid['본사정원수'] = (X_valid['본사정원수'] - TO_bias) / TO_scale
        #
        #if excess_scale != 0:
        #    X_train['본사시간외근무명령서승인건수'] = (X_train['본사시간외근무명령서승인건수'] - excess_bias) / excess_scale
        #    X_valid['본사시간외근무명령서승인건수'] = (X_valid['본사시간외근무명령서승인건수'] - excess_bias) / excess_scale
        #    
        #if home_scale != 0:
        #    X_train['현본사소속재택근무자수'] = (X_train['현본사소속재택근무자수'] - home_bias) / home_scale
        #    X_valid['현본사소속재택근무자수'] = (X_valid['현본사소속재택근무자수'] - home_bias) / home_scale
            
        if rain_scale != 0:
            X_train['강수량'] = (X_train['강수량'] - rain_bias) / rain_scale
            X_valid['강수량'] = (X_valid['강수량'] - rain_bias) / rain_scale    
            
        if reg:
            regmodel = sm.OLS(Y_train, X_train, missing='drop').fit()
        fitted_model = [models[i].fit(X_train, Y_train) for i in range(n_models)]         
                 
        
        #model1.fit(X_train, Y_train)
        #model2.fit(X_train, Y_train)
        y_pred=0
        for i in range(n_models):
            y_pred += weights[i] * fitted_model[i].predict(X_valid)
        
        if reg:
            y_pred += (1 - sum(weights)) * (regmodel.predict(X_valid))
        #y_pred = y_pred * train_x_lunch['본사출근자수'][test_index]
        #y_real = Y_valid * train_x_lunch['본사출근자수'][test_index]
        y_pred = y_pred * train_x_lunch['본사정원수'][test_index]
        y_real = Y_valid * train_x_lunch['본사정원수'][test_index]
        
        loss = np.mean(np.abs(y_real - y_pred))
        mae.append(loss)
    
    return mae, np.mean(mae)


def week_of_month(dt):

    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return str(int(np.ceil(adjusted_dom/7.0)))


def MunhwaDay(row):
    if len(row['석식메뉴']) < 20 or row['석식계'] == 0:
        return 1
    else:
        return 0

def CleanMenuList(row):
    if row['석식계'] == 0:
        return ''
    else:
        return row['석식메뉴']
        
    
def GetMenuList(row):
    row = row.split()
    result = []
    for food in row:
        if '(' in food and ')' in food and ':' in food:
            continue
        
        if '*' in food:
            result.extend(food.split('*'))
            continue
        
        food = re.sub('\(New\)', '', food)
        #food = re.sub('/\w+', '', food)
        
        result.append(food)
    return result


def TempClass(row):
    if row >= 30:
        return 'temp30+'
    elif row >= 25:
        return 'temp25+'
    elif row >= 20:
        return 'temp20+'
    elif row >= 15:
        return 'temp15+'
    elif row >= 10:
        return 'temp10+'
    elif row >= 5:
        return 'temp5+'
    elif row >= 0:
        return 'temp0+'
    elif row >= -5:
        return 'temp-5+'
    elif row >= -10:
        return 'temp-10+'
    elif row >= -15:
        return 'temp-15+'
    elif row >= -20:
        return 'temp-20+'
    else:
        return 'temp-20-'
    

    
def PreHolyday(row, pre_hol):
    if row in pre_hol.values:
        return 1
    else:
        return 0
    
def Temp30Class(row):
    if row >= 30:
        return 1
    else:
        return 0
    
    
def MenuEncoding(row, menu_list1, menu_list2):
    result = []
    for food in row:
        try:
            result.append(menu_list1[food])
        except:
            continue
    for food in row:
        try:
            result.append(menu_list2[food])
        except:
            continue
    
    
    return result

def MakeDTM(row):
    if len(row) == 0:
        return pd.Series(0, index=range(103))
    res = np.where(encoder.transform(np.array(row).reshape(-1,1)).toarray().sum(axis=0) >= 1, 1., 0.)
    return pd.Series(res)

In [90]:
train_data = pd.read_csv('train.csv')
train_data.index = pd.to_datetime(train_data['일자'])
del train_data['조식메뉴'], train_data['일자']
train_data.loc['2018-06-01']['요일'] = '금'


cases = pd.read_csv('일일코로나확진자.csv')
cases = cases[(cases['Entity'] == 'South Korea')].iloc[:, [2, 3]].reset_index(drop=True)
cases.index = pd.to_datetime(cases['Day'])
del cases['Day']
cases.columns = ['확진자수']
cases = cases[cases.index < '2021-04-10']
temp = cases.values.flatten().tolist()
temp.pop()
temp[0:0] = [0.]
cases['확진자수'] = temp
#cases = cases.rolling(5).mean().fillna(0)
#cases['확진자수'] = np.where(cases['확진자수'] == 0., 0., 1 / (1 + np.exp(- 1 / 100 * cases['확진자수'] + 3)))
#cases = cases.rolling(7).mean().fillna(0)
cases.columns = ['Covid19']
del temp

temp = pd.read_csv('진주시_기온정보.csv', encoding='euc-kr')
temp.index = pd.to_datetime(temp['날짜'])
del temp['날짜'], temp['지점'], temp['최저기온(℃)']

rain = pd.read_csv('TimeRain.csv')
rain.index = pd.to_datetime(rain['날짜'])
del rain['날짜'], rain['강수량']

dust = pd.read_csv('fine_dust.txt', sep='\t')
dust.index = pd.to_datetime(dust['날짜'])
del dust['날짜'], dust['PM2.5']


train_data = pd.concat([train_data, temp, cases, rain, dust], axis=1)
train_data = train_data[~train_data['본사정원수'].isnull()].fillna(0.)


    
train_data['개월'] = [f'{date.month}월' for date in train_data.index]
train_data['주차'] = [f'{week_of_month(date)}주차' for date in train_data.index]
train_data['문화day'] = train_data.apply(lambda row : MunhwaDay(row), axis=1)
train_data['석식메뉴'] = train_data.apply(lambda row : CleanMenuList(row), axis=1)

test_data = pd.read_csv('test.csv')
test_data.index = pd.to_datetime(test_data['일자'])
del test_data['조식메뉴'], test_data['일자']
test_data = pd.concat([test_data, temp, cases, rain, dust], axis=1)
test_data = test_data[~test_data['본사정원수'].isnull()].fillna(0.)
test_data['개월'] = [f'{date.month}월' for date in test_data.index]
test_data['주차'] = [f'{week_of_month(date)}주차' for date in test_data.index]
test_data['문화day'] = [0 for i in range(len(test_data))]




whole_df = pd.concat([train_data, test_data], axis=0)


#   
whole_df['본사출근자수'] = whole_df['본사정원수'] - whole_df['본사휴가자수'] - whole_df['현본사소속재택근무자수'] \
                         - whole_df['본사출장자수']

whole_df['본사휴가율'] = whole_df['본사휴가자수'] / whole_df['본사정원수']
whole_df['본사출장율'] = whole_df['본사출장자수'] / whole_df['본사정원수']
whole_df['본사재택근무율'] = whole_df['현본사소속재택근무자수'] / whole_df['본사정원수']

whole_df['시간외근무율'] = whole_df['본사시간외근무명령서승인건수'] / whole_df['본사출근자수']

whole_df['중식 참여율'] = whole_df['중식계'] / whole_df['본사정원수']#
whole_df['석식 참여율'] = whole_df['석식계'] / whole_df['본사정원수']#
whole_df['평균기온'] = whole_df['평균기온(℃)'].apply(TempClass)
whole_df['최고기온30+'] = whole_df['최고기온(℃)'].apply(Temp30Class)
whole_df['중식메뉴'] = whole_df['중식메뉴'].apply(GetMenuList)
whole_df['석식메뉴'] = whole_df['석식메뉴'].apply(GetMenuList)

del whole_df['본사휴가자수']#, whole_df['본사정원수']
del whole_df['현본사소속재택근무자수'], whole_df['본사출장자수']
del whole_df['본사시간외근무명령서승인건수']
del whole_df['중식계'], whole_df['석식계']
del whole_df['평균기온(℃)'], whole_df['최고기온(℃)']
    

                                                
pre_hol = pd.read_csv('pre_holiday.txt', encoding='utf8')
pre_hol = pd.to_datetime(pre_hol['공휴일전평일'])
whole_df['pre공휴일'] = [PreHolyday(date, pre_hol) for date in whole_df.index]
menu = whole_df[['중식메뉴', '석식메뉴']]
del whole_df['중식메뉴'], whole_df['석식메뉴']




##menu_label = pd.read_csv('menu_label.csv').fillna('없음')
#menu_label = pd.read_csv('menulabeling.csv').fillna('없음')
#menu_label['label1'] = menu_label['label1'].apply(lambda row : row.strip())
#menu_label['label2'] = menu_label['label2'].apply(lambda row : row.strip())
#menu_list1 = {}
#for _, row in menu_label[['menu', 'label1']].iterrows():
#    menu_list1[row['menu']] = row['label1']
#    
#menu_list2 = {}
#for _, row in menu_label[['menu', 'label2']].iterrows():
#    menu_list2[row['menu']] = row['label2']
##    
#menu['중식인코딩'] = menu['중식메뉴'].apply(lambda row : MenuEncoding(row, menu_list1, menu_list2))
#menu['석식인코딩'] = menu['석식메뉴'].apply(lambda row : MenuEncoding(row, menu_list1, menu_list2))
##
#encoder = OneHotEncoder()
#menu_list = menu_label['label1'].unique().tolist()
#menu_list.extend(menu_label['label2'].unique().tolist())
#
#menu_list = [[food] for food in menu_list]
#encoder.fit(menu_list)
##
#lunch_menu = menu['중식인코딩'].apply(MakeDTM)
#dinner_menu = menu['석식인코딩'].apply(MakeDTM)
#

day_dummy = pd.get_dummies(whole_df['요일'])
month_dummy = pd.get_dummies(whole_df['개월'])
week_dummy = pd.get_dummies(whole_df['주차'])
avgtemp_dummy = pd.get_dummies(whole_df[['평균기온']])

whole_df = pd.concat([whole_df, day_dummy, month_dummy, week_dummy,
                      avgtemp_dummy], axis=1)
drop_cols = ['요일', '개월', '주차', '평균기온']
whole_df.drop(drop_cols, axis=1, inplace=True)
whole_df = sm.add_constant(whole_df)
train_y_lunch = whole_df['중식 참여율'].dropna()
train_y_dinner = whole_df['석식 참여율'].dropna()
del whole_df['중식 참여율'], whole_df['석식 참여율']
whole_df['4주x금요일'] = whole_df['4주차'] * whole_df['금']
whole_df['timedelta'] = (np.array([(whole_df.index[i] - whole_df.index[i-1]).days for i in range(1, len(whole_df))] + [3]) - 1)





train_x, test_x = copy.deepcopy(whole_df.iloc[:len(train_data),:]), copy.deepcopy(whole_df.iloc[len(train_data):,:])
train_x.drop(datetime.datetime(2018,6,22), axis=0, inplace=True)
train_y_lunch.drop(datetime.datetime(2018,6,22), axis=0, inplace=True)
train_y_dinner.drop(datetime.datetime(2018,6,22), axis=0, inplace=True)

train_x_lunch, train_x_dinner = copy.deepcopy(train_x), copy.deepcopy(train_x)
del train_x_lunch['저녁강수량'], train_x_dinner['점심강수량']
test_x_lunch, test_x_dinner = copy.deepcopy(test_x), copy.deepcopy(test_x)
del test_x_lunch['저녁강수량'], test_x_dinner['점심강수량']
#



#train_x_lunch = pd.concat([train_x_lunch, lunch_menu], axis=1).dropna()
#test_x_lunch = pd.concat([test_x_lunch, lunch_menu], axis=1).dropna()
#train_x_dinner = pd.concat([train_x_dinner, dinner_menu], axis=1).dropna()
#test_x_dinner = pd.concat([test_x_dinner, dinner_menu], axis=1).dropna()

In [91]:
model1 = RandomForestRegressor(n_jobs=-1, random_state=42)
model2 = XGBRegressor(objective='reg:squarederror', n_estimators=100,
                      learning_rate=0.08, gamma=0, subsample=0.75,
                      colsample_bytree=1, max_depth=7, random_state=42)
model3 = Ridge(alpha=2.5, fit_intercept=True, random_state=42, normalize=False)
model4 = BaggingRegressor(random_state=30)
model5 = AdaBoostRegressor(random_state=42)
model6 = GradientBoostingRegressor(random_state=42)

In [88]:
k_fold_CrossValidation(train_x=train_x_lunch, 
                       train_y=train_y_lunch,
                       num=15,
                       models=[model1, model2, model3, model4, model6],
                       weights=[0.2, 0.2, 0.1, 0.1, 0.4],
                       minmax=False,
                       reg=False)
#output : 76.28

#model1 : 80.72
#model2 : 79.29
#model3 : 87.21
#model4 : 86.28
#model5: 98.07
#model6 : 78.92

([60.3636320934053,
  64.04428826941684,
  75.42639002663154,
  68.24943885676129,
  52.49862495895019,
  72.85318660096195,
  82.28386440728579,
  71.6695155215546,
  76.68914577499201,
  73.80093142575555,
  83.10250954151726,
  104.43454050205796,
  90.68977551357395,
  83.2174524706854,
  93.53876349304453],
 76.85747063043961)

In [89]:
k_fold_CrossValidation(train_x=train_x_dinner, 
                       train_y=train_y_dinner,
                       num=15,
                       models=[model1, model2, model4, model3, model6],
                       weights=[0.2, 0.2, 0.1, 0.1, 0.4],
                       minmax=False,
                       reg=False)
#output : 52.47


([49.65847346161556,
  48.69312180983775,
  51.5056032781334,
  49.4388011073643,
  38.25565141644013,
  51.22920797176222,
  46.07290659086086,
  49.69951298799195,
  46.589129393491746,
  46.64721840492652,
  64.67193775723375,
  57.256906843585135,
  71.25843798106185,
  53.167710434890786,
  64.70040875752639],
 52.58966854644816)

# 실제 구축

In [121]:
train_data = pd.read_csv('train.csv')
train_data.index = pd.to_datetime(train_data['일자'])
del train_data['조식메뉴'], train_data['일자']
train_data.loc['2018-06-01']['요일'] = '금'


cases = pd.read_csv('일일코로나확진자.csv')
cases = cases[(cases['Entity'] == 'South Korea')].iloc[:, [2, 3]].reset_index(drop=True)
cases.index = pd.to_datetime(cases['Day'])
del cases['Day']
cases.columns = ['확진자수']
cases = cases[cases.index < '2021-04-10']
temp = cases.values.flatten().tolist()
temp.pop()
temp[0:0] = [0.]
cases['확진자수'] = temp
#cases = cases.rolling(5).mean().fillna(0)
#cases['확진자수'] = np.where(cases['확진자수'] == 0., 0., 1 / (1 + np.exp(- 1 / 100 * cases['확진자수'] + 3)))
#cases = cases.rolling(7).mean().fillna(0)
cases.columns = ['Covid19']
del temp

temp = pd.read_csv('진주시_기온정보.csv', encoding='euc-kr')
temp.index = pd.to_datetime(temp['날짜'])
del temp['날짜'], temp['지점'], temp['최저기온(℃)']

rain = pd.read_csv('TimeRain.csv')
rain.index = pd.to_datetime(rain['날짜'])
del rain['날짜'], rain['강수량']

dust = pd.read_csv('fine_dust.txt', sep='\t')
dust.index = pd.to_datetime(dust['날짜'])
del dust['날짜'], dust['PM2.5']


train_data = pd.concat([train_data, temp, cases, rain, dust], axis=1)
train_data = train_data[~train_data['본사정원수'].isnull()].fillna(0.)


    
train_data['개월'] = [f'{date.month}월' for date in train_data.index]
train_data['주차'] = [f'{week_of_month(date)}주차' for date in train_data.index]
train_data['문화day'] = train_data.apply(lambda row : MunhwaDay(row), axis=1)
train_data['석식메뉴'] = train_data.apply(lambda row : CleanMenuList(row), axis=1)

test_data = pd.read_csv('test.csv')
test_data.index = pd.to_datetime(test_data['일자'])
del test_data['조식메뉴'], test_data['일자']
test_data = pd.concat([test_data, temp, cases, rain, dust], axis=1)
test_data = test_data[~test_data['본사정원수'].isnull()].fillna(0.)
test_data['개월'] = [f'{date.month}월' for date in test_data.index]
test_data['주차'] = [f'{week_of_month(date)}주차' for date in test_data.index]
test_data['문화day'] = [0 for i in range(len(test_data))]





whole_df = pd.concat([train_data, test_data], axis=0)


#   
whole_df['본사출근자수'] = whole_df['본사정원수'] - whole_df['본사휴가자수'] - whole_df['현본사소속재택근무자수'] \
                         - whole_df['본사출장자수']

whole_df['본사휴가율'] = whole_df['본사휴가자수'] / whole_df['본사정원수']
whole_df['본사출장율'] = whole_df['본사출장자수'] / whole_df['본사정원수']
whole_df['본사재택근무율'] = whole_df['현본사소속재택근무자수'] / whole_df['본사정원수']

whole_df['시간외근무율'] = whole_df['본사시간외근무명령서승인건수'] / whole_df['본사출근자수']

whole_df['중식 참여율'] = whole_df['중식계'] / whole_df['본사출근자수']##
whole_df['석식 참여율'] = whole_df['석식계'] / whole_df['본사출근자수']##
whole_df['평균기온'] = whole_df['평균기온(℃)'].apply(TempClass)
whole_df['최고기온30+'] = whole_df['최고기온(℃)'].apply(Temp30Class)
whole_df['중식메뉴'] = whole_df['중식메뉴'].apply(GetMenuList)
whole_df['석식메뉴'] = whole_df['석식메뉴'].apply(GetMenuList)

del whole_df['본사휴가자수'], whole_df['본사정원수']
del whole_df['현본사소속재택근무자수'], whole_df['본사출장자수']
del whole_df['본사시간외근무명령서승인건수']
del whole_df['중식계'], whole_df['석식계']
del whole_df['평균기온(℃)'], whole_df['최고기온(℃)']
    

                                                
pre_hol = pd.read_csv('pre_holiday.txt', encoding='utf8')
pre_hol = pd.to_datetime(pre_hol['공휴일전평일'])
whole_df['pre공휴일'] = [PreHolyday(date, pre_hol) for date in whole_df.index]
menu = whole_df[['중식메뉴', '석식메뉴']]
del whole_df['중식메뉴'], whole_df['석식메뉴']




#menu_label = pd.read_csv('menu_label.csv').fillna('없음')
menu_label = pd.read_csv('menulabeling.csv').fillna('없음')
menu_label['label1'] = menu_label['label1'].apply(lambda row : row.strip())
menu_label['label2'] = menu_label['label2'].apply(lambda row : row.strip())
menu_list1 = {}
for _, row in menu_label[['menu', 'label1']].iterrows():
    menu_list1[row['menu']] = row['label1']
    
menu_list2 = {}
for _, row in menu_label[['menu', 'label2']].iterrows():
    menu_list2[row['menu']] = row['label2']
#    
menu['중식인코딩'] = menu['중식메뉴'].apply(lambda row : MenuEncoding(row, menu_list1, menu_list2))
menu['석식인코딩'] = menu['석식메뉴'].apply(lambda row : MenuEncoding(row, menu_list1, menu_list2))
#
encoder = OneHotEncoder()
menu_list = menu_label['label1'].unique().tolist()
menu_list.extend(menu_label['label2'].unique().tolist())

menu_list = [[food] for food in menu_list]
encoder.fit(menu_list)
#
lunch_menu = menu['중식인코딩'].apply(MakeDTM)
dinner_menu = menu['석식인코딩'].apply(MakeDTM)


day_dummy = pd.get_dummies(whole_df['요일'])
month_dummy = pd.get_dummies(whole_df['개월'])
week_dummy = pd.get_dummies(whole_df['주차'])
avgtemp_dummy = pd.get_dummies(whole_df[['평균기온']])

whole_df = pd.concat([whole_df, day_dummy, month_dummy, week_dummy,
                      avgtemp_dummy], axis=1)
drop_cols = ['요일', '개월', '주차', '평균기온']
whole_df.drop(drop_cols, axis=1, inplace=True)
whole_df = sm.add_constant(whole_df)
train_y_lunch = whole_df['중식 참여율'].dropna()
train_y_dinner = whole_df['석식 참여율'].dropna()
del whole_df['중식 참여율'], whole_df['석식 참여율']
whole_df['4주x금요일'] = whole_df['4주차'] * whole_df['금']
whole_df['timedelta'] = (np.array([(whole_df.index[i] - whole_df.index[i-1]).days for i in range(1, len(whole_df))] + [3]) - 1)





train_x, test_x = copy.deepcopy(whole_df.iloc[:len(train_data),:]), copy.deepcopy(whole_df.iloc[len(train_data):,:])
train_x.drop(datetime.datetime(2018,6,22), axis=0, inplace=True)
train_y_lunch.drop(datetime.datetime(2018,6,22), axis=0, inplace=True)
train_y_dinner.drop(datetime.datetime(2018,6,22), axis=0, inplace=True)

to_bias = train_x['본사출근자수'].min()
to_scale = train_x['본사출근자수'].max() - train_x['본사출근자수'].min()
covid_bias = train_x['Covid19'].min()
covid_scale =  train_x['Covid19'].max() -  train_x['Covid19'].min()
dust10_bias = train_x['PM10'].min()
dust10_scale = train_x['PM10'].max() - train_x['PM10'].min()
#holy_bias = train_x['본사휴가자수'].mean()
#holy_scale = train_x['본사휴가자수'].std()
#out_bias = train_x['본사출장자수'].mean()
#out_scale = train_x['본사출장자수'].std()
#TO_bias = train_x['본사정원수'].min()
#TO_scale = train_x['본사정원수'].max() - train_x['본사정원수'].min()
#excess_bias = train_x['본사시간외근무명령서승인건수'].mean()
#excess_scale = train_x['본사시간외근무명령서승인건수'].std()
#home_bias = train_x['현본사소속재택근무자수'].mean()
#home_scale = train_x['현본사소속재택근무자수'].std()
lrain_bias = train_x['점심강수량'].min()
lrain_scale = train_x['점심강수량'].max() - train_x['점심강수량'].min()
drain_bias = train_x['저녁강수량'].min()
drain_scale = train_x['저녁강수량'].max() - train_x['저녁강수량'].min()

if to_scale != 0:
    train_x['본사출근자수'] = (train_x['본사출근자수'] - to_bias) / to_scale
    test_x['본사출근자수'] = (test_x['본사출근자수'] - to_bias) / to_scale


if covid_scale != 0:
    train_x['Covid19'] = (train_x['Covid19'] - covid_bias) / covid_scale
    test_x['Covid19'] = (test_x['Covid19'] - covid_bias) / covid_scale
    
if dust10_scale != 0:
    train_x['PM10'] = (train_x['PM10'] - dust10_bias) / dust10_scale
    test_x['PM10'] = (test_x['PM10'] - dust10_bias) / dust10_scale            
#    
#if holy_scale != 0:
#    train_x['본사휴가자수'] = (train_x['본사휴가자수'] - holy_bias) / holy_scale
#    test_x['본사휴가자수'] = (test_x['본사휴가자수'] - holy_bias) / holy_scale
#
#if out_scale != 0:
#    train_x['본사출장자수'] = (train_x['본사출장자수'] - out_bias) / out_scale
#    test_x['본사출장자수'] = (test_x['본사출장자수'] - out_bias) / out_scale
#
#if TO_scale != 0:
#    train_x['본사정원수'] = (train_x['본사정원수'] - TO_bias) / TO_scale
#    test_x['본사정원수'] = (test_x['본사정원수'] - TO_bias) / TO_scale
#
#if excess_scale != 0:
#    train_x['본사시간외근무명령서승인건수'] = (train_x['본사시간외근무명령서승인건수'] - excess_bias) / excess_scale
#    test_x['본사시간외근무명령서승인건수'] = (test_x['본사시간외근무명령서승인건수'] - excess_bias) / excess_scale
#    
#if home_scale != 0:
#    train_x['현본사소속재택근무자수'] = (train_x['현본사소속재택근무자수'] - home_bias) / home_scale
#    test_x['현본사소속재택근무자수'] = (test_x['현본사소속재택근무자수'] - home_bias) / home_scale
    
if lrain_scale != 0:
    train_x['점심강수량'] = (train_x['점심강수량'] - lrain_bias) / lrain_scale
    test_x['점심강수량'] = (test_x['점심강수량'] - lrain_bias) / lrain_scale   

if drain_scale != 0:
    train_x['저녁강수량'] = (train_x['저녁강수량'] - drain_bias) / drain_scale
    test_x['저녁강수량'] = (test_x['저녁강수량'] - drain_bias) / drain_scale   

    

train_x_lunch, train_x_dinner = copy.deepcopy(train_x), copy.deepcopy(train_x)
del train_x_lunch['저녁강수량'], train_x_dinner['점심강수량']
test_x_lunch, test_x_dinner = copy.deepcopy(test_x), copy.deepcopy(test_x)
del test_x_lunch['저녁강수량'], test_x_dinner['점심강수량']
#



train_x_lunch = pd.concat([train_x_lunch, lunch_menu], axis=1).dropna()
test_x_lunch = pd.concat([test_x_lunch, lunch_menu], axis=1).dropna()
train_x_dinner = pd.concat([train_x_dinner, dinner_menu], axis=1).dropna()
test_x_dinner = pd.concat([test_x_dinner, dinner_menu], axis=1).dropna()

In [123]:
sm.OLS(train_y_lunch, train_x_lunch).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 중식 참여율   R-squared:                       0.813
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     37.46
Date:                Tue, 06 Jul 2021   Prob (F-statistic):          1.11e-313
Time:                        01:40:50   Log-Likelihood:                 2268.1
No. Observations:                1204   AIC:                            -4284.
Df Residuals:                    1078   BIC:                            -3642.
Df Model:                         125                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.5854      0.025     23.082      0.000       0.536       0.635
Covid19           0.1164      0.014      8.455      0.000       0.089       0.143
점심강수량             0.0933      0.022      4.300      0.000       0.051       0.136
PM10             -0.0166      0.009     -1.848      0.065      -0.034       0.001
문화day            -0.0009      0.007     -0.117      0.907      -0.016       0.014
본사출근자수           -0.4555      0.017    -27.411      0.000      -0.488      -0.423
본사휴가율            -1.1819      0.051    -23.249      0.000      -1.282      -1.082
본사출장율            -2.4385      0.158    -15.416      0.000      -2.749      -2.128
본사재택근무율          -0.6353      0.062    -10.204      0.000      -0.757      -0.513
시간외근무율            0.2022      0.026      7.691      0.000       0.151       0.254
최고기온30+          -0.0026      0.006     -0.468      0.640      -0.013       0.008
pre공휴일           -0.0029      0.008     -0.362      0.717      -0.018       0.013
금                 0.0857      0.008     10.689      0.000       0.070       0.101
목                 0.0848      0.006     13.935      0.000       0.073       0.097
수                 0.1355      0.006     21.063      0.000       0.123       0.148
월                 0.1796      0.006     29.796      0.000       0.168       0.191
화                 0.0998      0.006     17.193      0.000       0.088       0.111
10월               0.0746      0.005     13.991      0.000       0.064       0.085
11월               0.0448      0.006      7.514      0.000       0.033       0.057
12월               0.0210      0.007      3.113      0.002       0.008       0.034
1월                0.0277      0.007      3.965      0.000       0.014       0.041
2월                0.0560      0.007      8.338      0.000       0.043       0.069
3월                0.0422      0.006      7.619      0.000       0.031       0.053
4월                0.0388      0.005      7.397      0.000       0.029       0.049
5월                0.0520      0.006      9.204      0.000       0.041       0.063
6월                0.0434      0.007      6.554      0.000       0.030       0.056
7월                0.0413      0.007      5.899      0.000       0.028       0.055
8월                0.0818      0.008     10.239      0.000       0.066       0.097
9월                0.0618      0.006      9.648      0.000       0.049       0.074
1주차               0.1035      0.006     18.103      0.000       0.092       0.115
2주차               0.1039      0.005     20.269      0.000       0.094       0.114
3주차               0.0994      0.005     19.503      0.000       0.089       0.109
4주차               0.0983      0.005     18.235      0.000       0.088       0.109
5주차               0.0980      0.005     18.529      0.000       0.088       0.108
6주차               0.0823      0.010      8.560   

In [124]:
model1_lunch = RandomForestRegressor(n_jobs=-1, random_state=42)
model2_lunch = XGBRegressor(objective='reg:squarederror', n_estimators=100,
                            learning_rate=0.08, gamma=0, subsample=0.75,
                            colsample_bytree=1, max_depth=7, random_state=42)
model3_lunch = model3 = Ridge(alpha=2.5, fit_intercept=True, random_state=42, normalize=False)
model4_lunch = BaggingRegressor(random_state=30)
model6_lunch = GradientBoostingRegressor(random_state=42)

model1_dinner = RandomForestRegressor(n_jobs=-1, random_state=42)
model2_dinner = XGBRegressor(objective='reg:squarederror', n_estimators=100,
                             learning_rate=0.08, gamma=0, subsample=0.75,
                             colsample_bytree=1, max_depth=7, random_state=42)
model3_dinner = model3 = Ridge(alpha=2.5, fit_intercept=True, random_state=42, normalize=False)
model4_dinner = BaggingRegressor(random_state=30)
model6_dinner = GradientBoostingRegressor(random_state=42)

In [125]:
model1_lunch.fit(train_x_lunch, train_y_lunch) 
model2_lunch.fit(train_x_lunch, train_y_lunch)    
model3_lunch.fit(train_x_lunch, train_y_lunch) 
model4_lunch.fit(train_x_lunch, train_y_lunch) 
model6_lunch.fit(train_x_lunch, train_y_lunch) 
model1_dinner.fit(train_x_dinner, train_y_dinner)
model2_dinner.fit(train_x_dinner, train_y_dinner)         
model3_dinner.fit(train_x_dinner, train_y_dinner)
model4_dinner.fit(train_x_dinner, train_y_dinner)
model6_dinner.fit(train_x_dinner, train_y_dinner)

GradientBoostingRegressor(random_state=42)

In [126]:
pred_lunch = 0.2 * model1_lunch.predict(test_x_lunch) + 0.3 * model2_lunch.predict(test_x_lunch) + \
             0.1 * model4_lunch.predict(test_x_lunch) + 0.4 * model6_lunch.predict(test_x_lunch)


pred_dinner = 0.2 * model1_dinner.predict(test_x_dinner) + 0.3 * model2_dinner.predict(test_x_dinner) + \
              0.1 * model4_dinner.predict(test_x_dinner) + 0.4 * model6_dinner.predict(test_x_dinner)


In [129]:
sub = pd.read_csv('sample_submission.csv')
sub['중식계'] = (pred_lunch * whole_df['본사출근자수'][-50:]).values.round()
sub['석식계'] = (pred_dinner * whole_df['본사출근자수'][-50:]).values.round()
sub.to_csv('submission_Ensemble_includingMenu_4models_NewFeatures.csv', index=False)

In [155]:
params_XGB = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror'],
              'learning_rate': [.03, 0.05, .08], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'subsample': [0.75],
              'colsample_bytree': [1],
              'n_estimators': [500],
              'random_state' : [42]}

params_RF = {
             "n_estimators" : [10,50,100],
             "max_features" : ["auto", "log2", "sqrt"],
             "bootstrap"    : [True, False],
             "random_state" : [42],
             "n_jobs" : [-1]
             }

params_BG = {'n_jobs' : [-1],
             'random_state' : [42]}
params_Ridge = {'alpha' : [2.5],
                'random_state' : [42],
                'normalize' : [False],
                'fit_intercept' : [True]}

params_GB =  {'learning_rate': [.03, 0.05, .08], #so called `eta` value
              'max_depth': [5, 6, 7],
              'n_estimators': [500],
              'random_state' : [42],
              'min_samples_leaf':[3], 
              'max_features':[1.0] }

model1 = RandomForestRegressor()
model2 = XGBRegressor()
model3 = Ridge()
model4 = BaggingRegressor()
model6 = GradientBoostingRegressor()


model1_lunch = GridSearchCV(model1, params_RF, scoring = 'neg_mean_absolute_error')

model2_lunch = GridSearchCV(model2, params_XGB, scoring='neg_mean_absolute_error')

model3_lunch = GridSearchCV(model3, params_Ridge, scoring='neg_mean_absolute_error')

model4_lunch = GridSearchCV(model4, params_BG, scoring = 'neg_mean_absolute_error')

model6_lunch = GridSearchCV(model6, params_GB, scoring = 'neg_mean_absolute_error')

model1_dinner = GridSearchCV(model1, params_RF, scoring = 'neg_mean_absolute_error')

model2_dinner = GridSearchCV(model2, params_XGB, scoring='neg_mean_absolute_error')

model3_dinner = GridSearchCV(model3, params_Ridge, scoring='neg_mean_absolute_error')

model4_dinner = GridSearchCV(model4, params_BG, scoring = 'neg_mean_absolute_error')

model6_dinner = GridSearchCV(model6, params_GB, scoring = 'neg_mean_absolute_error')


In [160]:
model1_lunch.fit(train_x_lunch, train_y_lunch)
print('done')
model2_lunch.fit(train_x_lunch, train_y_lunch)
model2_lunch = model2_lunch.best_estimator_
print('done')
model3_lunch.fit(train_x_lunch, train_y_lunch)
print('done')
model4_lunch.fit(train_x_lunch, train_y_lunch)
print('done')
model6_lunch.fit(train_x_lunch, train_y_lunch)


print('lunch done')

model1_dinner.fit(train_x_dinner, train_y_dinner)
print('done')
model2_dinner.fit(train_x_dinner, train_y_dinner)
model2_dinner = model2_dinner.best_estimator_
print('done')
model3_dinner.fit(train_x_dinner, train_y_dinner)
print('done')
model4_dinner.fit(train_x_dinner, train_y_dinner)
print('done')
model6_dinner.fit(train_x_dinner, train_y_dinner)

print('dinner done')

done
done
dinner done


In [167]:
pred_lunch = 0.2 * model1_lunch.predict(test_x_lunch) + 0.25 * model2_lunch.predict(test_x_lunch) + 0.05 * model3_lunch.predict(test_x_lunch) + 0.1 * model4_lunch.predict(test_x_lunch) + 0.4 * model6_lunch.predict(test_x_lunch)

pred_dinner = 0.2 * model1_dinner.predict(test_x_dinner) + 0.25 * model2_dinner.predict(test_x_dinner) + 0.05 * model3_dinner.predict(test_x_dinner) + 0.1 * model4_dinner.predict(test_x_dinner) + 0.4 * model6_dinner.predict(test_x_dinner)